In [8]:
import pandas as pd

In [9]:
def clean_flight_data(year):

    file = "raw_data/Flight_Details/" + str(year) + ".csv"

    original_df = pd.read_csv(file, sep=",", header=0, index_col=False,
                      usecols=[0,3,4,5,6,7,12,13,14,16,21,22,23,24,25,26],
                      na_filter = True)\
                     .reset_index(drop=True)\
                     .fillna(0)\
                     .astype({'CARRIER_DELAY': 'int32',
                              'WEATHER_DELAY': 'int32',
                              'NAS_DELAY': 'int32',
                              'SECURITY_DELAY': 'int32',
                              'LATE_AIRCRAFT_DELAY': 'int32'})
    # Streamline the Delay and Cancel information into 'weather' and 'not_weather' since that is all we care about.
    clean_df = original_df.loc[:]

    clean_df['Non Weather Delay'] = clean_df['CARRIER_DELAY']  + clean_df['NAS_DELAY'] + \
                                    clean_df['SECURITY_DELAY'] + clean_df['LATE_AIRCRAFT_DELAY']

    clean_df = clean_df.drop(columns=['CARRIER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY'])

    clean_df.loc[clean_df['CANCELLATION_CODE'] == 'B', 'CANCELLATION_CODE'] = 'weather'
    clean_df.loc[clean_df['CANCELLATION_CODE'] != 0, 'CANCELLATION_CODE'] = 'non weather'
    clean_df.loc[clean_df['CANCELLATION_CODE'] == 0, 'CANCELLATION_CODE'] = 'N/A'
    
    # Give columns more intuitive names
    clean_df = clean_df.rename(columns={'FL_DATE'          : 'Date',
                                        'ORIGIN'           : 'Origin',
                                        'DEST'             : 'Destination',
                                        'CRS_DEP_TIME'     : 'Expected Departure Time',
                                        'DEP_TIME'         : 'Actual Departure Time',
                                        'DEP_DELAY'        : 'Departure Delay',
                                        'CRS_ARR_TIME'     : 'Expected Arrival Time',
                                        'ARR_TIME'         : 'Actual Arrival Time',
                                        'ARR_DELAY'        : 'Arrival Delay',
                                        'DISTANCE'         : 'Distance Traveled',
                                        'CANCELLATION_CODE': 'Cancel Reason',
                                        'WEATHER_DELAY'    : 'Weather Delay'})
    return clean_df

In [13]:
for year in range(2009,2019):           ## Takes about 7 minutes ##
    output_path = 'clean_data/' + str(year) + '.csv'
    clean_flight_data(year).to_csv(output_path, index=True)